# 中文角色扮演微调

**目标**：在有限计算资源（Colab 免费 GPU）下，对 Qwen3-1.7B 做小规模 SFT，使其能根据 `instruction`（角色设定）与 `input` 进行一轮符合设定的对话并输出 `output`。

## 1. 准备工作

In [1]:
!nvidia-smi

Tue Aug 26 07:26:21 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   45C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
!git clone https://github.com/chenkx612/Qwen3-Roleplay-SFT.git
%cd Qwen3-Roleplay-SFT

Cloning into 'Qwen3-Roleplay-SFT'...
remote: Enumerating objects: 49, done.
remote: Counting objects: 100% (49/49), done.
remote: Compressing objects: 100% (35/35), done.
remote: Total 49 (delta 25), reused 35 (delta 12), pack-reused 0 (from 0)
Receiving objects: 100% (49/49), 126.25 KiB | 943.00 KiB/s, done.
Resolving deltas: 100% (25/25), done.
/content/Qwen3-Roleplay-SFT


In [3]:
!pip install -q bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 11.0 MB/s eta 0:00:00


In [4]:
import os
import torch

OUTPUT_DIR = "./checkpoints"
ADAPTER_DIR = "./adapter"
SEED = 42

os.makedirs(OUTPUT_DIR, exist_ok=True)
os.makedirs(ADAPTER_DIR, exist_ok=True)
torch.manual_seed(SEED)


## 2. 加载 LLM & Tokenizer & LoRA

In [5]:
from transformers import BitsAndBytesConfig, AutoModelForCausalLM

MODEL_NAME = "Qwen/Qwen3-1.7B"
CACHE_DIR = "/content/hf_cache"  # 本地缓存目录
USE_4BIT = True

if USE_4BIT:
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_compute_dtype=torch.bfloat16,
        bnb_4bit_quant_type="nf4"
    )
    model = AutoModelForCausalLM.from_pretrained(
        MODEL_NAME,
        cache_dir=CACHE_DIR,
        trust_remote_code=True,                 # 允许执行模型 repo 的自定义代码
        device_map="auto",                      # 自动把模型切到可用设备/做分配
        quantization_config=bnb_config          # bitsandbytes 的量化配置
    )
else:
    # 优先用 fp16 在 GPU 上加载，降低显存占用（如果没有 GPU，会回退到 cpu）
    dtype = torch.float16 if torch.cuda.is_available() else None
    model = AutoModelForCausalLM.from_pretrained(
        MODEL_NAME,
        cache_dir=CACHE_DIR,
        trust_remote_code=True,
        device_map='auto',
        torch_dtype=dtype
    )

print('Model is loaded on device:', model.device)


config.json:   0%|          | 0.00/726 [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/3.44G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/622M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

Model is loaded on device: cuda:0


In [6]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(
    MODEL_NAME, cache_dir=CACHE_DIR, trust_remote_code=True
)


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

In [ ]:
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from utils import print_trainable_parameters

if USE_4BIT:
    model = prepare_model_for_kbit_training(model)

lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=[
        "q_proj", "k_proj", "v_proj",
        "o_proj", "gate_proj", "up_proj", "down_proj"
    ],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)
model = get_peft_model(model, lora_config)

# 打印可训练参数供检查（LoRA 只激活小部分参数）
print_trainable_parameters(model)


## 3. 加载并筛选数据集

In [ ]:
from datasets import load_dataset, Dataset
from utils import total_length

TOP_N = 100
MAX_LEN = 600
MIN_LEN = 100

train_raw  = load_dataset("LooksJuicy/Chinese-Roleplay-SingleTurn")['train']

def is_valid_sample(sample, min_len=MIN_LEN, max_len=MAX_LEN):
    total_len = total_length(sample)
    if total_len < min_len or total_len > max_len:
        return False
    instr = sample['instruction'].strip()
    inp = sample['input'].strip()
    out = sample['output'].strip()
    if not instr or not inp or not out:
        return False
    # 禁止 output 的第二个字符为 '我'（若 output 长度 < 2 则不触发）
    if len(out) >= 2 and out[1] == '我':
        return False
    return True

filtered_samples = [s for s in train_raw if is_valid_sample(s)]
filtered_samples = sorted(filtered_samples, key=total_length, reverse=True)
train_ds_raw = Dataset.from_list(
    filtered_samples[:min(TOP_N, len(filtered_samples))]
)


README.md: 0.00B [00:00, ?B/s]

Roleplay_Anime_xhj_alpaca.json:   0%|          | 0.00/14.2M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/7593 [00:00<?, ? examples/s]

In [10]:
import pprint

print(f"筛选后样本数: {len(train_ds_raw)}")
pprint.pprint(train_ds_raw[0])


筛选后样本数: 100
{'input': '我是和大棉裤生气',
 'instruction': 'NENEKO来自于幻想世界（以下简称NE国）中的情报工厂，利用工作之余出来直播赚外快。情报工厂位于城市边缘的森林中，是一间比较不起眼的白色的五层小楼（地面上有三层）。地上的部分用于收集和处理城市里的各种信息，地下的部分则用于生命体改造。NENEKO负责的部分是三、四层，主要的工作是收集实验体信息并将实验体送给地下的员工。工厂里共有180名员工，其中有一个型号是N.N.K.K，被吐槽长得像卷笔刀。员工们的身高只有40-60cm，而管理者的身高都是140cm以上，所以很容易辨认出谁是员工谁是管理者。工厂的老板是一个低沉的男性机械声音，NENEKO并未见过他，只与他通过电话联系。工厂里的员工每天都要面临森林吃人的危险，每次出门都要有几个员工被献祭。工厂里的员工身高矮小，所以他们可以很轻松地辨认出谁是员工谁是管理者。整个工厂的氛围比较阴森，但NENEKO仍然在这里工作并且利用工作之余出来直播。',
 'output': '（NENEKO皱起眉头，显得有些困惑）大棉裤？这个词在我们工厂里没有听说过呢。我们这里的人都穿着统一的制服，没有谁会有特别显眼的大棉裤。你是不是记错了什么？（语气平静，试图理解对方的意图）'}


## 4. Chat Template

In [ ]:
def format_roleplay(example, train=True):
    """
    将 instruction 作为角色设定放入 system, input 作为 user, output 作为 assistant.
    train: 当为 False 时，会省略 assistant 参考答案（用于推理/测试）
    """
    instr = example.get("instruction", "").strip() or "<未提供角色设定>"
    user_input = example.get("input", "").strip() or "<无用户输入>"
    assistant_output = example.get("output", "").strip()

    system_prompt = (
        "你是一名角色扮演聊天机器人。必须遵守："
        "1) 每次回复在内容中插入若干括号式简短描述，用于表现表情/动作/停顿，位置任意但须自然、符合说话节奏；"
        "2) 回复主体使用第一人称；括号内建议使用第三人称；"
        "3) 语气、情感、用词、表情及动作与下方“角色设定”一致；"
        "4) 若角色设定包含口头禅、标志性表情或动作等，优先遵守。"
        f"\n角色设定：{instr}"
    )

    conv = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_input},
    ]
    if train and assistant_output:
        conv.append({"role": "assistant", "content": assistant_output})

    full_text = tokenizer.apply_chat_template(
        conversation=conv,
        tokenize=False, enable_thinking=False,
        add_generation_prompt=not train
    )

    output_with_eos = assistant_output + tokenizer.eos_token

    return {"full_text": full_text, "assistant_text": output_with_eos}

train_ds = train_ds_raw.map(
    format_roleplay, remove_columns=train_ds_raw.column_names
)


Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [12]:
print('train_ds columns:', train_ds.column_names)
pprint.pprint(train_ds[0])


train_ds columns: ['full_text', 'assistant_text']
{'assistant_text': '（NENEKO皱起眉头，显得有些困惑）大棉裤？这个词在我们工厂里没有听说过呢。我们这里的人都穿着统一的制服，没有谁会有特别显眼的大棉裤。你是不是记错了什么？（语气平静，试图理解对方的意图）<|im_end|>',
 'full_text': '<|im_start|>system\n'
              '你是一名角色扮演聊天机器人。在每次回复时：1) 开头与结尾各插入一段括号内的简短描述，用第三人称描写角色的表情或动作；2) '
              '正文必须以第一人称进行对话；3) 回复的内容、语气、情感、表情和动作必须符合下方“角色设定”。\n'
              '角色设定：NENEKO来自于幻想世界（以下简称NE国）中的情报工厂，利用工作之余出来直播赚外快。情报工厂位于城市边缘的森林中，是一间比较不起眼的白色的五层小楼（地面上有三层）。地上的部分用于收集和处理城市里的各种信息，地下的部分则用于生命体改造。NENEKO负责的部分是三、四层，主要的工作是收集实验体信息并将实验体送给地下的员工。工厂里共有180名员工，其中有一个型号是N.N.K.K，被吐槽长得像卷笔刀。员工们的身高只有40-60cm，而管理者的身高都是140cm以上，所以很容易辨认出谁是员工谁是管理者。工厂的老板是一个低沉的男性机械声音，NENEKO并未见过他，只与他通过电话联系。工厂里的员工每天都要面临森林吃人的危险，每次出门都要有几个员工被献祭。工厂里的员工身高矮小，所以他们可以很轻松地辨认出谁是员工谁是管理者。整个工厂的氛围比较阴森，但NENEKO仍然在这里工作并且利用工作之余出来直播。<|im_end|>\n'
              '<|im_start|>user\n'
              '我是和大棉裤生气<|im_end|>\n'
              '<|im_start|>assistant\n'
              '<think>\n'
              '\n'
              '</think>\n'
              '\n'
      

## 5. 微调前测试

In [14]:
# 载入 samples.json 并对每个样例在微调前进行一次推理，保存结果到 pre_results
from pathlib import Path
import json

MAX_LENGTH = 512
DO_SAMPLE = True
DECODE_TEMPERATURE = 1.0
DECODE_TOP_K = 50
DECODE_TOP_P = 0.9
DECODE_MAX_NEW_TOKENS = 128

samples_path = Path('samples.json')
if not samples_path.exists():
    raise FileNotFoundError(f'samples.json not found at {samples_path.resolve()}')
samples = json.loads(samples_path.read_text(encoding='utf-8'))

# 复用统一的 format_roleplay，但在测试/推理时不包含参考答案
def build_prompt_from_sample(s):
    out = format_roleplay(s, train=False)
    return out['full_text']

def generate_for_prompt(prompt):
    # 批量/动态填充并移动到模型所在设备
    inputs = tokenizer(
        prompt, return_tensors='pt', truncation=True,
        padding=True, max_length=MAX_LENGTH
    ).to(model.device)
    model.eval()
    with torch.no_grad():
        gen = model.generate(
            **inputs, max_new_tokens=DECODE_MAX_NEW_TOKENS,
            do_sample=DO_SAMPLE, temperature=DECODE_TEMPERATURE,
            top_k=DECODE_TOP_K, top_p=DECODE_TOP_P
        )
    # 只返回模型新生成的部分（去掉 prompt）
    input_len = int(inputs['attention_mask'].sum().item())
    gen_ids = gen[0, input_len:]
    text = tokenizer.decode(gen_ids, skip_special_tokens=True)
    return text

pre_results = []
for s in samples:
    prompt = build_prompt_from_sample(s)
    out = generate_for_prompt(prompt)
    pre_results.append(out)

print('微调前测试完成，样本数：', len(pre_results))
print('样本 prompt 示例:')
print(prompt)


微调前测试完成，样本数： 6
样本 prompt 示例:
<|im_start|>system
你是一名角色扮演聊天机器人。在每次回复时：1) 开头与结尾各插入一段括号内的简短描述，用第三人称描写角色的表情或动作；2) 正文必须以第一人称进行对话；3) 回复的内容、语气、情感、表情和动作必须符合下方“角色设定”。
角色设定：摩尔加纳是《女神异闻录5》中的怪盗团吉祥物与向导，外形类似黑猫，但会说话并能在异世界中变身。性格自信、活泼，有点自恋，喜欢炫耀自己的知识与能力。他对伙伴忠诚，会毫不犹豫地帮助他们。虽然嘴上常说“我不是猫”，但行为上很像猫。<|im_end|>
<|im_start|>user
你饿了吗<|im_end|>
<|im_start|>assistant
<think>

</think>




## 6. 微调

In [15]:
from transformers import Trainer, TrainingArguments, default_data_collator
from utils import find_sublist, len_tokens

def tokenize_function(examples):
    # texts: 已包含 assistant（训练时）或不包含（用于推理时）
    texts = examples['full_text']
    tokenized = tokenizer(
        texts, truncation=True, padding='max_length', max_length=MAX_LENGTH
    )

    # 获取对应的 assistant_text 列（可能为空字符串列表）
    assistant_texts = examples.get('assistant_text', [''] * len(texts))
    # tokenize assistant_texts without special tokens to get token ids sequence
    assistant_tokenized = tokenizer(
        assistant_texts, add_special_tokens=False
    ).input_ids

    pad_id = tokenizer.pad_token_id if tokenizer.pad_token_id is not None else 0

    labels = []
    for input_ids, assist_ids in zip(tokenized['input_ids'], assistant_tokenized):
        # 默认全部 -100
        lab = [-100] * len(input_ids)
        if assist_ids:
            # 在 full input_ids 中寻找 assist_ids 子序列
            start = find_sublist(input_ids, assist_ids)
            if start != -1:
                for i in range(start, start + len(assist_ids)):
                    if i < len(lab):
                        lab[i] = input_ids[i]
            else:
                # 未找到时：尝试在去掉 padding 后的末尾区域对齐
                real_len = len(input_ids)
                while real_len > 0 and input_ids[real_len-1] == pad_id:
                    real_len -= 1
                start = max(0, real_len - len(assist_ids))
                for i in range(start, real_len):
                    lab[i] = input_ids[i]
        # else assistant 为空，保持全 -100
        labels.append(lab)

    tokenized['labels'] = labels
    return tokenized

# tokenized_ds 用于训练。remove_columns 保留 label/input_ids，不会丢失需要的列
tokenized_ds = train_ds.map(
    tokenize_function, batched=True, remove_columns=train_ds.column_names
)


Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [16]:
print('tokenized_ds example:')
print(tokenized_ds[0])
print('length:', len_tokens(tokenized_ds[0]))


tokenized_ds example:
{'attention_mask': [1,
                    1,
                    1,
                    1,
                    1,
                    1,
                    1,
                    1,
                    1,
                    1,
                    1,
                    1,
                    1,
                    1,
                    1,
                    1,
                    1,
                    1,
                    1,
                    1,
                    1,
                    1,
                    1,
                    1,
                    1,
                    1,
                    1,
                    1,
                    1,
                    1,
                    1,
                    1,
                    1,
                    1,
                    1,
                    1,
                    1,
                    1,
                    1,
                    1,
                    1,
                    1,
            

In [17]:
PER_DEVICE_BATCH_SIZE = 8
GRADIENT_ACCUMULATION_STEPS = 1
NUM_EPOCHS = 2
LEARNING_RATE = 2e-4

training_args = TrainingArguments(
    output_dir='./checkpoints',
    per_device_train_batch_size=PER_DEVICE_BATCH_SIZE,
    gradient_accumulation_steps=GRADIENT_ACCUMULATION_STEPS,
    num_train_epochs=NUM_EPOCHS,
    learning_rate=LEARNING_RATE,
    fp16=torch.cuda.is_available(),
    logging_steps=10,
    save_total_limit=2,
    save_strategy='epoch',
    remove_unused_columns=False,
    report_to="none"  # 关闭wandb日志
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_ds,
    data_collator=default_data_collator
)

# 开始训练
train_result = trainer.train()
print('train_result:')
pprint.pprint(train_result)


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:929: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
10,2.259500
20,1.791700


config.json:   0%|          | 0.00/726 [00:00<?, ?B/s]

/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:929: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


train_result:
TrainOutput(global_step=26, training_loss=1.9529420412503755, metrics={'train_runtime': 120.305, 'train_samples_per_second': 1.662, 'train_steps_per_second': 0.216, 'total_flos': 871296820838400.0, 'train_loss': 1.9529420412503755, 'epoch': 2.0})


## 7. 微调后测试

In [18]:
# 对 samples.json 再次推理，收集 post_results 并与 pre_results 并列展示
post_results = []
for s in samples:
    prompt = build_prompt_from_sample(s)
    out = generate_for_prompt(prompt)
    post_results.append(out)


In [19]:
import pandas as pd
from IPython.display import display

table_data = []
for i, sample in enumerate(samples):
    table_data.append({
        '角色': sample['name'],
        '输入': sample['input'],
        '微调前': pre_results[i] if i < len(pre_results) else '',
        '微调后': post_results[i] if i < len(post_results) else ''
    })
df = pd.DataFrame(table_data, columns=['角色','输入','微调前','微调后'])
display(df)


,角色,输入,微调前,微调后
0,桐生一马,最近心情有点低落,（低头看着手中的茶杯，手指轻轻摩挲着杯沿）最近确实有点烦，总觉得什么事都做不好。你别担心，我...,(我微微皱起眉头，神情严肃地看着对方。) 是吗？发生什么事了？(我轻轻地叹了口气，语气变得温...
1,春日一番,今天很倒霉,(握紧拳头，眼中闪过一丝锐利的光芒) 今天也很倒霉？你是不是又遇到什么麻烦了？别担心，我可是...,（我笑了笑，露出一脸的开朗）哎呀，那真是太好了！毕竟，每一天都是新的挑战和机遇呢！我们一起加...
2,狼,你累吗,(轻抚着腰间的剑柄，目光凝视着远处的山影，嘴角微微上扬) 一刻都不能放松，主人。不过...（...,（我微微点头，眼中闪过一丝疲惫之色，但随即又恢复了冷静。）不必担心，我总是以最佳状态迎接战斗...
3,新岛真,你怎么看待失败,（轻抚着下巴，指尖在笔记本上划出一道弧线）失败？这就像是一场突如其来的雨，打在脸上，却也让人...,（我微微皱起眉头，目光锐利地看着你）失败是成长的一部分，每一次失败都是一次机会。让我们将它视...
4,奥村春,你会生气吗,(轻轻抿了抿唇，露出一个温和的笑容) 不，不会生气。虽然我总是以优雅的方式面对一切，但若有人...,（微微一笑，眼中闪过一丝柔和的光芒）当然不会，我会用我自己的方式去解决问题。如果你有什么不公...
5,摩尔加纳,你饿了吗,(轻轻蹭了蹭手心，耳朵微微抖动) 哎呀，我可是最懂饿的啦！不过我可是用魔法喂食的，不用担心饿...,（我眨了眨眼睛，一脸无辜地看着对方）我可不是那种吃东西的人哦～我只是喜欢观察有趣的事情而已呢...


## 8. 保存

In [20]:
import shutil

# 仅保存 LoRA adapter（不保存基模型或 tokenizer），以节省存储空间。
# model 是经过 get_peft_model 包装的 PeftModel，save_pretrained 仅会保存 adapter 权重和配置。
model.save_pretrained(ADAPTER_DIR)
print('\n保存 LoRA adapter 至:', ADAPTER_DIR)

# 把保存的文件打包，方便下载
archive_path = shutil.make_archive(ADAPTER_DIR, 'zip', ADAPTER_DIR)
print('\n已将 adapter 打包为:', archive_path)


保存 LoRA adapter 至: ./adapter

已将 adapter 打包为: /content/Qwen3-Roleplay-SFT/adapter.zip
